In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
#!pip install tensorflow
#!pip install tflearn
import tflearn
#!pip3 install tensorflow==1.2.0 --ignore-installed
import tensorflow as tf
import random
import json
import string
import unicodedata
import sys
import pandas as pd

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:

# a table structure to hold the different punctuation used
tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                    if unicodedata.category(chr(i)).startswith('P'))

# method to remove punctuations from sentences.
def remove_punctuation(text):
    return text.translate(tbl)

# initialize the stemmer
stemmer = LancasterStemmer()
# variable to hold the Json data read from the file
#data = None


In [ ]:

# read the json file and load the training data
with open('data.json') as json_data:
    data = json.load(json_data)
    print(data)



In [13]:
dados = pd.read_csv('~/Documents/dsa/data/reviewFinal.csv',encoding='latin-1')
corpus = pd.DataFrame()
listTitle = []
listRecom = []

for i, r in dados.iterrows():
    if r['recommend'] != 'NI' and r['title'] != "" and r['title'] is not np.nan and r['site'] == "glassdoor":
        listTitle.append(r['title'])
        listRecom.append(r['recommend'])


In [14]:
corpus['title'] = listTitle
corpus['recommend'] = listRecom
data = corpus[:5000]
len(data)

5000

In [5]:
# get a list of all categories to train for
categories = list(set(listRecom))
words = []
# a list of tuples with words in the sentence and category name
docs = []

#for each_category in data.keys():
for i, row in data.iterrows():
    # remove any punctuation from the sentence
    each_sentence = remove_punctuation(row['title'])

    #print(each_sentence)

    # extract words from each sentence and append to the word list
    w = nltk.word_tokenize(each_sentence)
    #print("tokenized words: ", w)

    words.extend(w)
    docs.append((w, row['recommend']))


In [6]:

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words]
words = sorted(list(set(words)))

#print(words)
#print(docs)

In [7]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(categories)


In [8]:
for doc in docs:
    # initialize our bag of words(bow) for each document in the list
    bow = []
    # list of tokenized words for the pattern
    token_words = doc[0]
    # stem each word
    token_words = [stemmer.stem(word.lower()) for word in token_words]
    # create our bag of words array
    for w in words:
        bow.append(1) if w in token_words else bow.append(0)

    output_row = list(output_empty)
    output_row[categories.index(doc[1])] = 1

    # our training set will contain a the bag of words model and the output row that tells
    # which catefory that bow belongs to.
    training.append([bow, output_row])


In [9]:
# shuffle our features and turn into np.array as tensorflow  takes in numpy array
random.shuffle(training)
training = np.array(training)

# trainX contains the Bag of words and train_y contains the label/ category
train_x = list(training[:, 0])
train_y = list(training[:, 1])


In [12]:

# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
#model.save('model.tflearn')


Training Step: 624999  | total loss: 0.23649 | time: 2.566s
| Adam | epoch: 1000 | loss: 0.23649 - acc: 0.8807 -- iter: 4992/5000
Training Step: 625000  | total loss: 0.21429 | time: 2.570s
| Adam | epoch: 1000 | loss: 0.21429 - acc: 0.8927 -- iter: 5000/5000
--


In [15]:
def get_tf_record(sentenceL):
    #print(sentenceL)
    global words
    retorno = []
    if type(sentenceL) == list :
        for sentence in sentenceL:
            #print(sentence)
            # tokenize the pattern
            sentence_words = nltk.word_tokenize(sentence)
            # stem each word
            sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
            # bag of words
            bow = [0]*len(words)
            for s in sentence_words:
                for i, w in enumerate(words):
                    if w == s:
                        bow[i] = 1
            retorno.append(bow)

        return(np.array(retorno))
    else:
        sentence = sentenceL
        # tokenize the pattern
        sentence_words = nltk.word_tokenize(sentence)
        # stem each word
        sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
        # bag of words
        bow = [0]*len(words)
        for s in sentence_words:
            for i, w in enumerate(words):
                if w == s:
                    bow[i] = 1
        retorno.append(bow)

        return(np.array(bow))
        

In [57]:
#output_row[categories.index(doc[1])] = 1
categories.index(x['recommend'])

1

In [60]:
data_test = corpus[10000:20000]
title_test = []
y_test = []
for i,x in data_test.iterrows():
    title_test.append(x['title'])
    y_test.append(categories.index(x['recommend']))
    
title_test = get_tf_record(title_test)
pred = model.predict(title_test)
pred_y = list(np.argmax(pred,axis=1))
#pred.max(axis=0)
pred_y
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred_y)

0.66

In [ ]:
# let's test the mdodel for a few sentences:
# the first two sentences are used for training, and the last two sentences are not present in the training data.
sent_1 = "what time is it?"
sent_2 = "I gotta go now"
sent_3 = "do you know the time now?"
sent_4 = "you must be a couple of years older then her!"

sents = ["what time is it?","I gotta go now","do you know the time now?","you must be a couple of years older then her!"]

In [ ]:
print(np.argmax(model.predict([get_tf_record(sent_1)])))
categories

In [ ]:

# we can start to predict the results for each of the 4 sentences
print(categories[np.argmax(model.predict([get_tf_record(sent_1)]))])
print(categories[np.argmax(model.predict([get_tf_record(sent_2)]))])
print(categories[np.argmax(model.predict([get_tf_record(sent_3)]))])
print(categories[np.argmax(model.predict([get_tf_record(sent_4)]))])

In [ ]:
x = get_tf_record(sents)
labels = [0,3,3,4]


In [ ]:
pred = model.predict(x)

In [ ]:
print(pred)
#print(categories[np.argmax(model.predict([get_tf_record(sent_1)]))])
#print(model.predict([get_tf_record(sent_1)]))
#print(np.argmax(get_tf_record([sent_1])))
#print(get_tf_record(sent_1))

In [ ]:
pred_y = list(np.argmax(pred,axis=1))
#pred.max(axis=0)
pred_y

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(labels,pred_y)
#print(labels, pred_y)